In [14]:
import tensorflow as tf
from tensorflow.keras import layers, Input, Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.datasets import mnist

# 모델 생성 함수
def get_mnist_model():
    model = Sequential([
        layers.Flatten(input_shape=(28 * 28,)),  # 입력을 784 차원으로 설정
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

# 모델 초기화
model = get_mnist_model()
loss_fn = SparseCategoricalCrossentropy()
optimizer = RMSprop()
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
loss_tracking_metric = tf.keras.metrics.Mean()

# 훈련 함수 정의
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs[metric.name] = metric.result()
    loss_tracking_metric.update_state(loss)
    logs['loss'] = loss_tracking_metric.result()
    return logs

# 메트릭 초기화 함수
def reset_metrics():
    for metric in metrics:
        metric.reset_state()
    loss_tracking_metric.reset_state()

# 데이터 로드 및 전처리
(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

# 데이터셋 준비
training_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(32)

# 훈련 루프
epochs = 3
for epoch in range(epochs):
    reset_metrics()
    for inputs_batch, targets_batch in training_dataset:
        logs = train_step(inputs_batch, targets_batch)
    print(f"{epoch + 1}번째 에포크 결과")
    for key, value in logs.items():
        print(f"...{key}: {value:.4f}")

# 단계별 평가 루프
@tf.function
def test_step(inputs, targets):
    predictions = model(inputs, training=False)
    loss = loss_fn(targets, predictions)

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs['val_'+metric.name] = metric.result()
    loss_tracking_metric.update_state(loss)
    logs['val_loss'] = loss_tracking_metric.result()
    return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()
for inputs_batch, targets_batch in val_dataset:
    logs = test_step(inputs_batch, targets_batch)
print('평가 결과:')
for key, value in logs.items():
    print(f'...{key}: {value:.4f}')

1번째 에포크 결과
...sparse_categorical_accuracy: 0.9245
...loss: 0.2591
2번째 에포크 결과
...sparse_categorical_accuracy: 0.9668
...loss: 0.1143
3번째 에포크 결과
...sparse_categorical_accuracy: 0.9768
...loss: 0.0814
평가 결과:
...val_sparse_categorical_accuracy: 0.9640
...val_loss: 0.1363


In [ ]:
# 사용자 정의 훈련 스텝 구현
loss_fn = SparseCategoricalCrossentropy()
loss_tracker = tf.keras.metrics.Mean(name='loss')

class CustomModel(tf.keras.Model):
    def train_step(self, data):
        inputs, targets = data
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = loss_fn(targets, predictions)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        loss_tracker.update_state(loss)
        return {'loss': loss_tracker.result()}
    
    @property
    def metrics(self):
        return [loss_tracker]

# 모델 정의
inputs = Input(shape=(28 * 28,))
features = layers.Dense(512, activation='relu')(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation='softmax')(features)
model = CustomModel(inputs, outputs)

# 컴파일 및 훈련
model.compile(optimizer=RMSprop())  
model.fit(train_images, train_labels, epochs=3)



Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.4483
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.1635
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.1300


In [ ]:
# 사용자 정의 훈련 스텝이 포함된 모델 클래스
class CustomModel(tf.keras.Model):
    def train_step(self, data):
        inputs, targets = data
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.compiled_loss(targets, predictions)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        self.compiled_metrics.update_state(targets, predictions)

        return {m.name: m.result() for m in self.metrics}

# 모델 정의
inputs = Input(shape=(28 * 28,))
features = layers.Dense(512, activation='relu')(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation='softmax')(features)
model = CustomModel(inputs, outputs)

# 컴파일 및 훈련
model.compile(
    optimizer=RMSprop(),  
    loss=SparseCategoricalCrossentropy(),  # 손실 함수 설정
    metrics=[SparseCategoricalAccuracy()]   # 정확도 메트릭 설정
)

# 훈련
model.fit(train_images, train_labels, epochs=3)


Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - sparse_categorical_accuracy: 0.8628 - loss: 0.1000
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - sparse_categorical_accuracy: 0.9533 - loss: 0.1000
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - sparse_categorical_accuracy: 0.9609 - loss: 0.1000


In [19]:
# 깊은 층과 얕은 층(layer)

# 얕은 레이어 모델 생성 함수
def get_shallow_mnist_model():
    model = Sequential([
        layers.Flatten(input_shape=(28 * 28,)),  # 입력을 784 차원으로 설정
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

# 깊은 레이어 모델 생성 함수
def get_deep_mnist_model():
    model = Sequential([
        layers.Flatten(input_shape=(28 * 28,)),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

# 데이터 로드 및 전처리
(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

# 데이터셋 준비
training_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(32)

# 공통 설정
loss_fn = SparseCategoricalCrossentropy()
optimizer = RMSprop()
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
loss_tracking_metric = tf.keras.metrics.Mean()

# 메트릭 초기화 함수
def reset_metrics():
    for metric in metrics:
        metric.reset_state()
    loss_tracking_metric.reset_state()

# 훈련 함수 정의
def train_step(model, inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs[metric.name] = metric.result()
    loss_tracking_metric.update_state(loss)
    logs['loss'] = loss_tracking_metric.result()
    return logs

# 평가 함수 정의
@tf.function
def test_step(model, inputs, targets):
    predictions = model(inputs, training=False)
    loss = loss_fn(targets, predictions)

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs['val_'+metric.name] = metric.result()
    loss_tracking_metric.update_state(loss)
    logs['val_loss'] = loss_tracking_metric.result()
    return logs

# 모델 훈련 및 평가 루프
def train_and_evaluate(model, optimizer, epochs=3):
    print(f"모델 구조: {model.summary()}")
    
    for epoch in range(epochs):
        reset_metrics()
        for inputs_batch, targets_batch in training_dataset:
            logs = train_step(model, optimizer, inputs_batch, targets_batch)  # optimizer 전달
        print(f"{epoch + 1}번째 에포크 결과")
        for key, value in logs.items():
            print(f"...{key}: {value:.4f}")

    reset_metrics()
    for inputs_batch, targets_batch in val_dataset:
        logs = test_step(model, inputs_batch, targets_batch)
    print('평가 결과:')
    for key, value in logs.items():
        print(f'...{key}: {value:.4f}')
    print("\n" + "="*40 + "\n")

# 훈련 함수 정의 - optimizer를 추가 매개변수로 전달
def train_step(model, optimizer, inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs[metric.name] = metric.result()
    loss_tracking_metric.update_state(loss)
    logs['loss'] = loss_tracking_metric.result()
    return logs

# 얕은 레이어 모델 훈련 및 평가
print("얕은 레이어 모델 훈련 및 평가")
shallow_model = get_shallow_mnist_model()
shallow_optimizer = RMSprop()  # 얕은 모델용 옵티마이저
train_and_evaluate(shallow_model, shallow_optimizer)

# 깊은 레이어 모델 훈련 및 평가
print("깊은 레이어 모델 훈련 및 평가")
deep_model = get_deep_mnist_model()
deep_optimizer = RMSprop()  # 깊은 모델용 옵티마이저
train_and_evaluate(deep_model, deep_optimizer)



얕은 레이어 모델 훈련 및 평가


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_7 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 64)             │        50,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,890 (198.79 KB)

 Trainable params: 50,890 (198.79 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_8 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 244,522 (955.16 KB)

 Trainable params: 244,522 (955.16 KB)

 Non-trainable params: 0 (0.00 B)

모델 구조: None
1번째 에포크 결과
...sparse_categorical_accuracy: 0.9256
...loss: 0.2489
2번째 에포크 결과
...sparse_categorical_accuracy: 0.9693
...loss: 0.1083
3번째 에포크 결과
...sparse_categorical_accuracy: 0.9792
...loss: 0.0764
평가 결과:
...val_sparse_categorical_accuracy: 0.9605
...val_loss: 0.1631




------------------------------
+ 얕은 레이어 모델: 3 에포크 동안 훈련한 결과, 최종 훈련 정확도는 약 96.09%, 손실은 0.1363으로 나타났으며, 검증 정확도는 95.69%, 검증 손실은 0.1486

+ 깊은 레이어 모델: 3 에포크 동안 훈련한 결과, 최종 훈련 정확도는 약 97.92%, 손실은 0.0764로 향상되었으며, 검증 정확도는 96.05%, 검증 손실은 0.1631

## 요약: 깊은 레이어 모델이 얕은 레이어 모델보다 훈련 정확도와 손실에서 약간 더 좋은 성능을 보였으나, 검증 손실은 얕은 모델이 조금 더 낮게 나타남
--------------------------------

In [20]:
#학습률 차이(높음과 낮음)

# 모델 생성 함수
def get_mnist_model():
    model = Sequential([
        layers.Flatten(input_shape=(28 * 28,)),  # 입력을 784 차원으로 설정
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

# 공통 설정
loss_fn = SparseCategoricalCrossentropy()
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
loss_tracking_metric = tf.keras.metrics.Mean()

# 메트릭 초기화 함수
def reset_metrics():
    for metric in metrics:
        metric.reset_state()
    loss_tracking_metric.reset_state()

# 훈련 함수 정의
def train_step(model, optimizer, inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs[metric.name] = metric.result()
    loss_tracking_metric.update_state(loss)
    logs['loss'] = loss_tracking_metric.result()
    return logs

# 평가 함수 정의
@tf.function
def test_step(model, inputs, targets):
    predictions = model(inputs, training=False)
    loss = loss_fn(targets, predictions)

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs['val_'+metric.name] = metric.result()
    loss_tracking_metric.update_state(loss)
    logs['val_loss'] = loss_tracking_metric.result()
    return logs

# 모델 훈련 및 평가 루프
def train_and_evaluate(learning_rate, epochs=3):
    print(f"학습률: {learning_rate}")
    model = get_mnist_model()
    optimizer = RMSprop(learning_rate=learning_rate)
    
    for epoch in range(epochs):
        reset_metrics()
        for inputs_batch, targets_batch in training_dataset:
            logs = train_step(model, optimizer, inputs_batch, targets_batch)
        print(f"{epoch + 1}번째 에포크 결과")
        for key, value in logs.items():
            print(f"...{key}: {value:.4f}")

    reset_metrics()
    for inputs_batch, targets_batch in val_dataset:
        logs = test_step(model, inputs_batch, targets_batch)
    print('평가 결과:')
    for key, value in logs.items():
        print(f'...{key}: {value:.4f}')
    print("\n" + "="*40 + "\n")

# 데이터 로드 및 전처리
(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

# 데이터셋 준비
training_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(32)

# 높은 학습률로 훈련 및 평가
print("높은 학습률로 훈련 및 평가")
train_and_evaluate(learning_rate=0.1)

# 낮은 학습률로 훈련 및 평가
print("낮은 학습률로 훈련 및 평가")
train_and_evaluate(learning_rate=0.0001)


높은 학습률로 훈련 및 평가
학습률: 0.1
1번째 에포크 결과
...sparse_categorical_accuracy: 0.4953
...loss: 2.8643
2번째 에포크 결과
...sparse_categorical_accuracy: 0.4195
...loss: 1.5607
3번째 에포크 결과
...sparse_categorical_accuracy: 0.3789
...loss: 1.6172
평가 결과:
...val_sparse_categorical_accuracy: 0.3653
...val_loss: 1.5467


낮은 학습률로 훈련 및 평가
학습률: 0.0001
1번째 에포크 결과
...sparse_categorical_accuracy: 0.8318
...loss: 0.6626
2번째 에포크 결과
...sparse_categorical_accuracy: 0.9181
...loss: 0.2852
3번째 에포크 결과
...sparse_categorical_accuracy: 0.9337
...loss: 0.2320
평가 결과:
...val_sparse_categorical_accuracy: 0.9359
...val_loss: 0.2219




----------------------
+ 높은 학습률 (0.1): 훈련 과정에서 불안정한 성능을 보였으며, 최종 훈련 정확도는 **37.89%**로 낮고, 손실은 1.6172로 높음. 검증 정확도는 36.53%, 검증 손실은 1.5467로 나타나 과적합이 제대로 이루어지지 않았음.

+ 낮은 학습률 (0.0001): 훈련이 안정적으로 진행되었으며, 최종 훈련 정확도는 91.81%, 손실은 0.2852로 양호한 성능을 보임. 검증 정확도는 약 90% 이상으로 평가 결과도 좋았음.

## 요약: 낮은 학습률이 더 안정적이며, 정확도와 손실 측면에서 높은 학습률보다 훨씬 우수한 성능을 보임.
-------------------------

In [21]:
# 모델 생성 함수 (기본 구조 유지)
def get_mnist_model(activation='relu'):
    model = Sequential([
        layers.Flatten(input_shape=(28 * 28,)),  # 입력을 784 차원으로 설정
        layers.Dense(128, activation=activation),
        layers.Dense(64, activation=activation),
        layers.Dense(10, activation='softmax')
    ])
    return model

# 데이터 로드 및 전처리
(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

# 데이터셋 준비
training_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(32)

# 공통 설정
loss_fn = SparseCategoricalCrossentropy()
metrics = [SparseCategoricalAccuracy()]
loss_tracking_metric = tf.keras.metrics.Mean()

# 메트릭 초기화 함수
def reset_metrics():
    for metric in metrics:
        metric.reset_state()
    loss_tracking_metric.reset_state()

# 훈련 함수 정의
def train_step(model, optimizer, inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs[metric.name] = metric.result()
    loss_tracking_metric.update_state(loss)
    logs['loss'] = loss_tracking_metric.result()
    return logs

# 평가 함수 정의
@tf.function
def test_step(model, inputs, targets):
    predictions = model(inputs, training=False)
    loss = loss_fn(targets, predictions)

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs['val_' + metric.name] = metric.result()
    loss_tracking_metric.update_state(loss)
    logs['val_loss'] = loss_tracking_metric.result()
    return logs

# 모델 훈련 및 평가 루프
def train_and_evaluate(model, optimizer, epochs=3):
    print(f"모델 구조: {model.summary()}")
    
    for epoch in range(epochs):
        reset_metrics()
        for inputs_batch, targets_batch in training_dataset:
            logs = train_step(model, optimizer, inputs_batch, targets_batch)
        print(f"{epoch + 1}번째 에포크 결과")
        for key, value in logs.items():
            print(f"...{key}: {value:.4f}")

    reset_metrics()
    for inputs_batch, targets_batch in val_dataset:
        logs = test_step(model, inputs_batch, targets_batch)
    print('평가 결과:')
    for key, value in logs.items():
        print(f'...{key}: {value:.4f}')
    print("\n" + "="*40 + "\n")

# tanh 활성화 함수 모델 훈련 및 평가
print("tanh 활성화 함수 모델 훈련 및 평가")
tanh_model = get_mnist_model(activation='tanh')
tanh_optimizer = RMSprop()
train_and_evaluate(tanh_model, tanh_optimizer)

# sigmoid 활성화 함수 모델 훈련 및 평가
print("sigmoid 활성화 함수 모델 훈련 및 평가")
sigmoid_model = get_mnist_model(activation='sigmoid')
sigmoid_optimizer = RMSprop()
train_and_evaluate(sigmoid_model, sigmoid_optimizer)


tanh 활성화 함수 모델 훈련 및 평가


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_11 (Flatten)            │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_56 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,386 (427.29 KB)

 Trainable params: 109,386 (427.29 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_12 (Flatten)            │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_57 (Dense)                │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_58 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_59 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,386 (427.29 KB)

 Trainable params: 109,386 (427.29 KB)

 Non-trainable params: 0 (0.00 B)

모델 구조: None
1번째 에포크 결과
...sparse_categorical_accuracy: 0.8711
...loss: 0.5034
2번째 에포크 결과
...sparse_categorical_accuracy: 0.9385
...loss: 0.2078
3번째 에포크 결과
...sparse_categorical_accuracy: 0.9531
...loss: 0.1551
평가 결과:
...val_sparse_categorical_accuracy: 0.9520
...val_loss: 0.1578




-----------------------
### tanh 활성화 함수 모델:

+ 최종 훈련 정확도: 97.20%
+ 최종 훈련 손실: 0.0955
+ 검증 정확도: 96.36%
+ 검증 손실: 0.1216
--------------

### sigmoid 활성화 함수 모델:

+ 최종 훈련 정확도: 95.31%
+ 최종 훈련 손실: 0.1551
+ 검증 정확도: 95.20%
+ 검증 손실: 0.1578
--------------
요약: tanh 활성화 함수를 사용한 모델이 sigmoid를 사용한 모델보다 훈련 및 검증 정확도와 손실에서 더 우수한 성능을 보임. 기울기가 tanh가 더 높기 때문.
------------

------------
회고
============
# 점점 속도가 빨라진다.
## 자꾸 해보니 모델 구조가 조금씩 눈에 익어간다.
### 모델이 뭘 학습하고 뱉어내는지 감만 잡힌다.
-----------